## Command and interrupt class 2(Graph control flow)
*** Operations with Interrupt ***
- To replace traditional "Input" function running in the terminal
- Human in the loop
- Resume
- Update and Resume
- Rewind/time Travel
- Branch
- Abort

In [10]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Image, display
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

In [11]:
memory = MemorySaver()

class State(TypedDict):
    text: str

In [12]:
def node_a(state:State):
    print("Node A")
    return Command(
        goto="node_b",
        update={"text": state["text"] + "a"}
    )

def node_b(state:State):
    print("Node B")

    human_response = interrupt("Do you want to go to C or D? ..to answer type C/D: ")
    print("Human review values: ", human_response)

    if(human_response == "C"):
        return Command(
            goto="node_c",
            update={"text": state["text"] + "b"}
        )
    elif (human_response == "D"):
        return Command(
            goto="node_d",
            update={"text": state["text"] + "b"}
        )

def node_c(state:State):
    print("Node C")
    return Command(
        goto=END,
        update={"text": state["text"] + "c"}
    )

def node_d(state:State):
    print("Node D")
    return Command(
        goto=END,
        update={"text": state["text"] + "d"}
    )

In [13]:
graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}
initial_state={"text":""}
# display(Image(app.get_graph().draw_mermaid_png()))

In [14]:
first_result = app.invoke(initial_state, config=config, stream_mode="values")
first_result

Node A
Node B


{'text': 'a',
 '__interrupt__': [Interrupt(value='Do you want to go to C or D? ..to answer type C/D: ', resumable=True, ns=['node_b:ff0cef39-c7bc-36ec-b973-cd8a95c80bee'])]}

In [15]:
print(app.get_state(config).next)

('node_b',)


In [16]:
second_result = app.invoke(Command(resume="D"), config, stream_mode="updates")
second_result

Node B
Human review values:  D
Node D


[{'node_b': {'text': 'ab'}}, {'node_d': {'text': 'abd'}}]